## Question

The dataset contains text of online travel reviews (in Column Review) with an associated Rating (column Overall_Rating). 
The objective is to train a classifier to predict the rating from the Review text. 
You are free to choose the model's architecture, but you should describe and justify your design choices.
Train the model and assess it as appropriate in machine learning. You are allowed to preprocess the data however you want 
(e.g. using pretrained embeddings, dropping some features, just a bag-of-words), but the predictive model must be trained 
by yourself from scratch (no pretrained predictor). 

### Problem definition
- ***Dataset***: Online travel reviews with their corresponding ratings
- ***Inputs***: Travel reviews
- ***Output***: Predict the rating from the review

We are training a classifier to predict the rating from the review text

### Step by step.
- **Load data**
    - *loading the data*
    - *splitting the data into train and test*
- **Data preprocessing**
    - **Text cleaning:** *Remove noise(pactuation, stopwords)*
    - **Text normalization:** *Lowercasing, Stemming, Lemmantization*
    - **Tokenization:** *Split text into words, or subwords, or characters*
- **Feature extraction/embeddings**
    - *Bag of words*
    - *TF-IDF*
- **Model Selection**
    - *Naive Bayes classifier*
- **Model Training**
    - *Train on the tain dataset*
    - *Monitor loss and accuracy on validation dataset*
    - **Techniques**
        - *Hyperparameter tuning*
        - *Cross validation*
        - *Regularization(dropout, weight decay)*
    - *Train on the tain dataset*
- **Model evaluation**
    - **Metrics**
        - *Accuracy, precision, recall, f1 score*
        - *Confusion matrix*

In [ ]:
## Installing some libraries and packages.


In [1]:
## Importing neccessary libraries